<a href="https://colab.research.google.com/github/nhn09/faqSimilarity/blob/main/FAQtask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np 
import pandas as pd

In [3]:
Dataset = pd.read_csv('/content/FAQs.csv', delimiter = ",")
Dataset.head(10)

,Question,Answer
0,When was Albert Einstein born?,Albert Einstein was born on 14 March 1879.
1,Where was he born?,"He was born in Ulm, Germany."
2,When did he die?,"He died 18 April 1955 in Princeton, New Jersey..."
3,Who were his parents?,His father was Hermann Einstein and his mother...
4,Did he have any sisters and brothers?,He had one sister named Maja.
5,Did he marry and have children?,He was married to Mileva Marić between 1903 an...
6,Where did he receive his education?,He received his main education at the followin...
7,When was Albert Einstein awarded the Nobel Pri...,"The Nobel Prize Awarding Institution, the Roya..."
8,Did Albert Einstein attend the Nobel Prize Awa...,The Nobel Prize was announced on 9 November 19...
9,For what did he receive the Nobel Prize?,Einstein was rewarded for his many contributio...


# Embedding Using SBERT:

SentenceTransformers is a Python framework for state-of-the-art sentence, text and image embeddings.

The framework is based on PyTorch and Transformers.

In [4]:
!pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 3.5 MB/s 
     |████████████████████████████████| 5.5 MB 48.0 MB/s 
     |████████████████████████████████| 1.3 MB 28.6 MB/s 
     |████████████████████████████████| 163 kB 43.9 MB/s 
     |████████████████████████████████| 7.6 MB 44.0 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=2e90754569e750fbfe866beee97185a9423c950d9f3c35e925afc56a2e19d9ce
  Stored in directory: /root/.cache/pip/wheels/bf/06/fb/d59c1e5bd1dac7f6cf61ec0036cc3a10ab8fecaa6b2c3d3ee9
Successfully built sentence-transformers


In [5]:
testDataset = pd.read_csv('/content/FAQs_test.csv', delimiter = ",")
testDataset.head()

,Question
0,What is the date of his death?
1,Did Einstein have siblings?
2,Who was his wife?
3,What was Einstein's father's name?
4,At what institutions did he study?


In [12]:
testSentences = testDataset['Question'].values.tolist()
givenSentences = Dataset['Question'].values.tolist()
givenAnsSentences = Dataset['Answer'].values.tolist()
print(testSentences)
print(givenSentences)
print(givenAnsSentences)

['What is the date of his death?', 'Did Einstein have siblings?', 'Who was his wife?', "What was Einstein's father's name?", 'At what institutions did he study?']
['When was Albert Einstein born?', 'Where was he born?', 'When did he die?', 'Who were his parents?', 'Did he have any sisters and brothers?', 'Did he marry and have children?', 'Where did he receive his education?', 'When was Albert Einstein awarded the Nobel Prize in Physics?', 'Did Albert Einstein attend the Nobel Prize Award Ceremony?', 'For what did he receive the Nobel Prize?']
['Albert Einstein was born on 14 March 1879.', 'He was born in Ulm, Germany. ', 'He died 18 April 1955 in Princeton, New Jersey, USA.', 'His father was Hermann Einstein and his mother was Pauline Einstein (born Koch).', 'He had one sister named Maja.', 'He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived

In [36]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')

sentences = testSentences
embeddings1 = model.encode(testSentences, convert_to_tensor=True)
embeddings2 = model.encode(givenSentences, convert_to_tensor=True)
cosine_scores = util.cos_sim(embeddings1, embeddings2)

In [38]:
for i in range(len(testSentences)): 
  for j in range(len(givenSentences)): 
   print("{} \t\t {} \t\t Score: {:.4f}".format(testSentences[i], givenSentences[j], cosine_scores[i][j]))

What is the date of his death? 		 When was Albert Einstein born? 		 Score: 0.4319
What is the date of his death? 		 Where was he born? 		 Score: 0.5214
What is the date of his death? 		 When did he die? 		 Score: 0.8528
What is the date of his death? 		 Who were his parents? 		 Score: 0.4387
What is the date of his death? 		 Did he have any sisters and brothers? 		 Score: 0.3483
What is the date of his death? 		 Did he marry and have children? 		 Score: 0.3962
What is the date of his death? 		 Where did he receive his education? 		 Score: 0.3921
What is the date of his death? 		 When was Albert Einstein awarded the Nobel Prize in Physics? 		 Score: 0.2773
What is the date of his death? 		 Did Albert Einstein attend the Nobel Prize Award Ceremony? 		 Score: 0.3315
What is the date of his death? 		 For what did he receive the Nobel Prize? 		 Score: 0.3400
Did Einstein have siblings? 		 When was Albert Einstein born? 		 Score: 0.7280
Did Einstein have siblings? 		 Where was he born? 		 Sc

In [39]:
for i in range(len(testSentences)): 
  list_cosine = list( cosine_scores[i].numpy())
  max_value = np.max(list_cosine)
  ansIndex = list_cosine.index(max_value)
  print("{} \t\t {} \t\t {} \t\t Score: {:.4f}".format(testSentences[i], givenSentences[ansIndex], givenAnsSentences[ansIndex],  cosine_scores[i][ansIndex]))

What is the date of his death? 		 When did he die? 		 He died 18 April 1955 in Princeton, New Jersey, USA. 		 Score: 0.8528
Did Einstein have siblings? 		 When was Albert Einstein born? 		 Albert Einstein was born on 14 March 1879. 		 Score: 0.7280
Who was his wife? 		 Did he marry and have children? 		 He was married to Mileva Marić between 1903 and 1919. They had three children, Lieserl (born 1902), Hans Albert (born 1904) and Eduard (born 1910). He married Elsa Löwenthal in 1919 and they lived together until her death in 1936. 		 Score: 0.6313
What was Einstein's father's name? 		 When was Albert Einstein born? 		 Albert Einstein was born on 14 March 1879. 		 Score: 0.7501
At what institutions did he study? 		 Where did he receive his education? 		 He received his main education at the following schools: Catholic elementary school in Munich, Germany (1885-1888)Luitpold Gymnasium in Munich, Germany (1888-1894) Cantonal school in Aarau, Switzerland (1895-1896) Swiss Federal Institute 